In [1]:
import functools

import einops
import equinox as eqx
import jax
import jax.numpy as jnp
import numpy as np
import optax

from models.slaternet import SlaterNet
from models.psi_solid import PsiSolid

from systems.continuous import moire

key = jax.random.PRNGKey(42)

\begin{equation}
    H=\sum_i\left(-\frac{1}{2}\nabla_i^2 + V(\mathbf r_i)\right) + \sum_{i < j}\frac{1}{|\mathbf r_i-\mathbf r_j|}
\end{equation}
- Moire potential:
\begin{equation}
    V(\mathbf r) = -2V_0\sum_{j=0}^2 \cos(\mathbf g_j\cdot\mathbf r + \varphi)
\end{equation}
Where $\mathbf g_j=\frac{4\pi}{\sqrt{3}a_M}\left(\cos\frac{2\pi j}{3},\sin\frac{2\pi j}{3}\right)$.

- Units
  - mass: Electron mass, $m_e$
  - Length: Effective Bohr radius, $a_0^* = \epsilon\dfrac{m_e}{m^*} a_0$
  - Energy: Effective Hartree, $\mathrm{Ha}^* = \dfrac{1}{\epsilon^2}\dfrac{m^*}{m_e}\mathrm{Ha}$

In [ ]:
# system parameters
a_0 = 5.291_772_105_44e-11 # m
m_e = 9.109_383_7139e-31 # kg
Ha = 4.359_744_722_2060e-18 # J = kg m^2/s^2
meV = 1.60218e-22 # J = kg m^2/s^2
nm = 1e-9 # m

epsilon = 5
# epsilon = 10
m_eff = 0.35 * m_e
a_B_eff = epsilon * (m_e / m_eff) * a_0
Ha_eff = (1 / (epsilon ** 2)) * (m_eff / m_e) * Ha


V0 = 15 * meV / Ha_eff
a_M = 8.031 * nm / a_B_eff
varphi = jnp.pi / 4
(V0, a_M, varphi)

(0.0393743564807352, 10.623473362015782, 0.7853981633974483)

In [ ]:
primitive_latt_vects = a_M * jnp.array([
    [jnp.cos(-jnp.pi / 2), jnp.sin(-jnp.pi / 2)],
    [jnp.cos(jnp.pi / 6), jnp.sin(jnp.pi / 6)]
]) # [a_0, a_1]

primitive_recip_latt_vects = ((4 * jnp.pi) / (jnp.sqrt(3) * a_M)) * jnp.array([
    [jnp.cos(-jnp.pi / 3), jnp.sin(-jnp.pi / 3)],
    [jnp.cos(0), jnp.sin(0)]
]) # [b_0, b_1]
# a_i \cdot b_j = 2 \pi \delta_{ij}

(min1, max1) = (-3, 2)
(min2, max2) = (-3, 2)

(minu, maxu) = (min1 + 1/2, max1 + 1/2)
(minv, maxv) = (min2 + 1/2, max2 + 1/2)

# parametrization:
u, v = 0.1234, 1.1245
R = u * primitive_latt_vects[0] + v * primitive_latt_vects[1]

# super cell: L_x := (max1 - min1) by L_y := (max2 - min2) cell, L_x L_y atoms)
from math import ceil
G = primitive_recip_latt_vects * jnp.array([
    1 / ceil(max1 - min1),
    1 / ceil(max2 - min2),
])

In [ ]:
subkey, key = jax.random.split(key)

slater_wavefn = SlaterNet(
    num_particle=18, 
    recip_latt_vecs=moire.G, 
    hidden_dim=64,
    mlp_depth=8,
    key=subkey,
)

In [3]:
subkey, key = jax.random.split(key)

psi_solid_wavefn = PsiSolid(
    num_particle=18, 
    recip_latt_vecs=moire.G, 
    hidden_dim=64,
    attention_dim=16,
    num_heads=6,
    num_blocks=3, 
    num_det=4,
    key=subkey,
)